In [1]:
import cv2
import os
import numpy as np
import xml.etree.ElementTree as ET
from pathlib import Path

In [2]:
def is_within(box1, box2):
  """
  Verifica se a box1 está contida dentro da box2
  """
  xtl1, ytl1, xbr1, ybr1= box1
  xtl2, ytl2, xbr2, ybr2 = box2
  
  return xtl1 <= xtl2 and xbr1 >= xbr2 and ytl1 <= ytl2 and ybr1 >= ybr2

In [3]:
def parse_box(element):
  """
  Analisa um elemento de caixa e retorna as coordenadas
  """
  xtl = float(element.get("xtl"))
  ytl = float(element.get("ytl"))
  xbr = float(element.get("xbr"))
  ybr = float(element.get("ybr"))
  return xtl, ytl, xbr, ybr

In [4]:
class X:
  def __init__(self, xtl, ytl, xbr, ybr):
    self.xtl = xtl
    self.ytl = ytl
    self.xbr = xbr
    self.ybr = ybr
    self.heads = []
  
  def __str__(self):
    return f"({self.xtl}, {self.ytl}, {self.xbr}, {self.ybr})"

class Heads:
  def __init__(self, xtl, ytl, xbr, ybr):
    self.xtl = xtl
    self.ytl = ytl
    self.xbr = xbr
    self.ybr = ybr
    self.eyes = []
    
  def __str__(self):
    return f"({self.xtl}, {self.ytl}, {self.xbr}, {self.ybr})"


In [5]:
def normalize_coords(x_box, sub_box):
    """
    Normaliza as coordenadas da sub_box em relação à x_box.
    """
    xtl1, ytl1, xbr1, ybr1 = parse_box(x_box)
    xtl2, ytl2, xbr2, ybr2 = parse_box(sub_box)

    # Ajusta as coordenadas da sub_box em relação à x_box
    xtl = xtl2 - xtl1
    ytl = ytl2 - ytl1
    xbr = xbr2 - xtl1
    ybr = ybr2 - ytl1

    return int(xtl), int(ytl), int(xbr), int(ybr)

In [6]:
def calculate_area(box):
  """
  Calcula a área de uma caixa dada suas coordenadas (xtl, ytl, xbr, ybr).
  """
  xtl, ytl, xbr, ybr = box
  return (xbr - xtl) * (ybr - ytl)

In [7]:
def check_boxes_by_list_of_boxes_and_crop(boxes, path):
    """
    Verifica quais caixas de 'Head' e 'Eye' estão dentro das caixas 'X' e realiza o recorte.
    Apenas a cabeça que ocupa a maior área dentro de 'X' será considerada.
    A cabeça será pintada de branco e a área ao redor será preta, com os olhos pintados de vermelho.
    """
    head_boxes = boxes.get("Head", [])
    eye_boxes = boxes.get("Eye", [])
    x_boxes = boxes.get("X", [])
    index = 0
    print(f"Quantidade de caixas 'X': {len(x_boxes)}")
    
    image = cv2.imread(path)
    
    # Verifica se a imagem foi carregada corretamente
    if image is None:
        print(f"Erro ao carregar a imagem: {path}")
        return
    
    for x_box in x_boxes:
        index += 1
        
        base_name = os.path.basename(path).replace('.png', '')
        dir_name = os.path.dirname(path)
        cropped_path = os.path.join(dir_name + "/x", f"{base_name}_x{index}.png")
        
        # Extrai as coordenadas da caixa X e converte para inteiros
        xlt, ylt, xbr, ybr = map(int, parse_box(x_box))
        
        # Recorta a área 'X' usando OpenCV
        cropped_image = image[ylt:ybr, xlt:xbr]
        
        y_image = cropped_image.copy()
        
        # Seleciona a cabeça que ocupa a maior área dentro da caixa 'X'
        max_head_area = 0
        selected_head_box = None
        
        for head_box in head_boxes:
            if is_within(parse_box(x_box), parse_box(head_box)):
                head_area = calculate_area(parse_box(head_box))
                if head_area > max_head_area:
                    max_head_area = head_area
                    selected_head_box = head_box
        
        if selected_head_box:
            xtl_head, ytl_head, xbr_head, ybr_head = normalize_coords(x_box, selected_head_box)
            # Pinta tudo de preto primeiro
            y_image[:] = 0
            # Desenha um retângulo branco ao redor da cabeça
            cv2.rectangle(y_image, (xtl_head, ytl_head), (xbr_head, ybr_head), (255, 255, 255), -1)
            
            for eye_box in eye_boxes:
                if is_within(parse_box(selected_head_box), parse_box(eye_box)):
                    xtl_eye, ytl_eye, xbr_eye, ybr_eye = normalize_coords(x_box, eye_box)
                    # Desenha um retângulo vermelho ao redor dos olhos
                    cv2.rectangle(y_image, (xtl_eye, ytl_eye), (xbr_eye, ybr_eye), (0, 0, 255), -1)
        
        y_dir = os.path.join(dir_name + "/y") 
        if not os.path.exists(y_dir):
            os.makedirs(y_dir)
        
        resized_y = cv2.resize(y_image, (128, 128))
        
        cv2.imwrite(os.path.join(y_dir, f"{base_name}_y{index}.png"), resized_y)
        
        # Redimensiona a imagem para 128x128 pixels
        resized_image = cv2.resize(cropped_image, (128, 128))
            
        # Verifica se o diretório existe, caso contrário, cria-o
        output_dir = os.path.dirname(cropped_path)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            
        # Salva a imagem recortada
        cv2.imwrite(cropped_path, resized_image)

In [8]:
def extract_image_data(xml_file, image_path):
    """
    Extrai as caixas de 'Head', 'Eye' e 'X' do arquivo XML e realiza o recorte da imagem.
    """
    tree = ET.parse(xml_file)
    root = tree.getroot()



    for image_element in root.iter("image"):
        image_name = image_element.attrib.get('name')
        
            # Initialize the dictionary for image boxes
        images_boxes = {
            'Head': [],
            'Eye': [],
            'X': []
        }
        if image_name:
            full_image_path = os.path.join(image_path, image_name)
            
            if os.path.isfile(full_image_path):
                for box in image_element.iter("box"):
                    attr = box.attrib
                    label = attr.get('label')
                    if label in images_boxes:
                        images_boxes[label].append(attr)
                
                # Call the function with the dictionary
                check_boxes_by_list_of_boxes_and_crop(images_boxes, full_image_path)
            else:
                print(f"Imagem não encontrada: {full_image_path}")

In [9]:

# Example usage
"""
Modo de uso:
1. Carregar o arquivo XML - modifique o caminho do arquivo XML
2. Carregar o diretório de imagens - modifique o caminho do diretório de imagens
3. Chamar a função extract_image_data passando por parâmetro o arquivo XML e o diretório de imagens
"""
xml_file = "../dataset/29-07-24/annotations_7813_9724/annotations_7813_9724.xml"
image_path = "../dataset/29-07-24/annotations_7813_9724"

data = extract_image_data(xml_file, image_path)

Quantidade de caixas 'X': 2
Quantidade de caixas 'X': 4
Quantidade de caixas 'X': 7
Quantidade de caixas 'X': 9
Quantidade de caixas 'X': 10
Quantidade de caixas 'X': 12
Quantidade de caixas 'X': 14
Quantidade de caixas 'X': 17
Quantidade de caixas 'X': 19
Quantidade de caixas 'X': 22
Quantidade de caixas 'X': 24
Quantidade de caixas 'X': 26
Quantidade de caixas 'X': 27
Quantidade de caixas 'X': 29
Quantidade de caixas 'X': 31
Quantidade de caixas 'X': 34
Quantidade de caixas 'X': 36
Quantidade de caixas 'X': 37
Quantidade de caixas 'X': 38
Quantidade de caixas 'X': 40
Quantidade de caixas 'X': 42
Quantidade de caixas 'X': 44
Quantidade de caixas 'X': 45
Quantidade de caixas 'X': 46
Quantidade de caixas 'X': 49
Quantidade de caixas 'X': 50
Quantidade de caixas 'X': 51
Quantidade de caixas 'X': 52
Quantidade de caixas 'X': 53
Quantidade de caixas 'X': 54
Quantidade de caixas 'X': 55
Quantidade de caixas 'X': 56
Quantidade de caixas 'X': 57
Quantidade de caixas 'X': 58
Quantidade de caix